In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Văn bản tiêu đề mặc định
!pip install transformers
!pip install -U sentence-transformers
!pip install pyvi
!pip install rank-bm25
!pip install gdown
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn.functional as F
from rank_bm25 import BM25Okapi
import json
from datetime import datetime
import gdown
import zipfile
import os
import re
from gensim.utils import simple_preprocess
import json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=b3bbc632ff8457d41b4976f84df3bb526bcde38047853f0f105f3e206cccbd24
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 57.9 MB/s e

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def preprocess_text(text: str) -> str:
    text = re.sub(r"['\",\.\?:\-!]", "", text)
    text = text.strip()
    text = " ".join(text.split())
    text = text.lower()
    return text
def preprocess(sentence):
    text1 = ' '.join(simple_preprocess(sentence))
    return text1

In [ ]:
df2 = pd.read_json('/content/drive/MyDrive/Dataset/output72.json')
evidences = []
claims = []
ids = []
i = 0
for key, value in df2.items():
    claim = value['claim']
    evidence = value["evidence"]
    if (preprocess_text(evidence) == preprocess_text(claim)):
      i = i + 1
print(i)

25


In [ ]:
#Load data
df = pd.read_json('/content/drive/MyDrive/ise-dsc01-private-test-offcial.json')
passages = []
claims = []
ids = []
for key, value in df.items():
    claim = value['claim']
    passage = value['context']
    claims.append(claim)
    ids.append(key)
    passages.append(passage)

In [ ]:
model2 = AutoModel.from_pretrained("sentence-transformers/xlm-r-distilroberta-base-paraphrase-v1")
tokenizer2 = AutoTokenizer.from_pretrained("sentence-transformers/xlm-r-distilroberta-base-paraphrase-v1")

In [ ]:
# Create a list of dictionaries to store the data for each claim
output_data = {}
for passage, claim, id in zip(passages, claims, ids):
    # premise_sentences = sent_tokenize(passage)
      claim = str(claim)
      passage = re.sub(r'\n', '. ', passage)

      premise_sentences = re.split(r'(?<=[.!?])\s+', passage)
      # Filter out empty strings and remove leading/trailing whitespace
      premise_sentences = [s.strip() for s in premise_sentences if s.strip()]
      premise_sentences = [preprocess_text(sentence) for sentence in premise_sentences]
      #BM25 rerank top k
      tokenized_premise_sentences = [word.split(" ") for word in premise_sentences]
      tokenized_claim = claim.split(" ")
      bm25 = BM25Okapi(tokenized_premise_sentences)
      bm25_scores = bm25.get_scores(tokenized_claim)
      top_k_sentences = bm25.get_top_n(tokenized_claim, premise_sentences, n=5)
      # print(top_k_sentences)
      encoded_claim2 = tokenizer2(claim, return_tensors='pt')
      encoded_premise2 = [tokenizer2(preprocess(sentence), return_tensors='pt') for sentence in top_k_sentences]
      embedded_premise2 = [model2(**sentence) for sentence in encoded_premise2]
      embedded_claim2 = model2(**encoded_claim2)

      highest_similarity2 = -1.1
      best_premise_sentence2 = None

      # Tính cos sim từng câu trong top k để so sánh với claim
      for sentence, premise_embedding2 in zip(top_k_sentences, embedded_premise2):

          similarity2 = F.cosine_similarity(embedded_claim2.last_hidden_state.mean(dim=1),
                                            premise_embedding2.last_hidden_state.mean(dim=1), dim=1)
          # print(similarity2)

          if similarity2 > highest_similarity2:
              highest_similarity2 = similarity2
              best_premise_sentence2 = preprocess_text(sentence)
      if(best_premise_sentence2 == claim):
          premise_sentences.remove(best_premise_sentence2)
          #BM25 rerank top k
          tokenized_premise_sentences = [word.split(" ") for word in premise_sentences]
          tokenized_claim = claim.split(" ")
          bm25 = BM25Okapi(tokenized_premise_sentences)
          bm25_scores = bm25.get_scores(tokenized_claim)
          top_k_sentences = bm25.get_top_n(tokenized_claim, premise_sentences, n=5)
          # print(top_k_sentences)
          encoded_claim2 = tokenizer2(claim, return_tensors='pt')
          encoded_premise2 = [tokenizer2(preprocess(sentence), return_tensors='pt') for sentence in top_k_sentences]
          embedded_premise2 = [model2(**sentence) for sentence in encoded_premise2]
          embedded_claim2 = model2(**encoded_claim2)

          highest_similarity2 = -1.1
          best_premise_sentence2 = None

          # Tính cos sim từng câu trong top k để so sánh với claim
          for sentence, premise_embedding2 in zip(top_k_sentences, embedded_premise2):

              similarity2 = F.cosine_similarity(embedded_claim2.last_hidden_state.mean(dim=1),
                                                premise_embedding2.last_hidden_state.mean(dim=1), dim=1)
              # print(similarity2)

              if similarity2 > highest_similarity2:
                  highest_similarity2 = similarity2
                  best_premise_sentence2 = preprocess_text(sentence)
      claim_data = {
            "claim": claim,
            "evidence": best_premise_sentence2
      }
      print(claim_data)

      output_data[id] = claim_data
print('finish-task')
with open('/content/drive/MyDrive/Dataset/output.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(output_data, jsonfile, indent=4)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
{'claim': 'Có 1 con gấu rất dễ thương đã uống rượu rum mạnh trộn cùng với cặn của ara, một loại rượu ủ tại địa phương, ở làng Babesa của Bhutan vào ngày 3/1', 'evidence': 'con gấu đói đã uống rượu rum mạnh trộn cùng với cặn của ara một loại rượu ủ tại địa phương ở làng babesa của bhutan hôm 3/1'}
{'claim': 'Các chuyên gia khuyến cáo, trước khi chạy, nhất là ở cự ly chạy dài, runner nên ăn những thức ăn có thể tiêu nhanh như dưa hấu, vải...', 'evidence': 'các chuyên gia khuyến cáo trước khi chạy nhất là ở cự ly chạy dài runner nên ăn những thức ăn có độ kết dính cao như cơm trắng chuối'}
{'claim': 'Trên khuôn mặt ông Bình có nét đẹp hiền từ nên được nhiều người yêu mến', 'evidence': 'sự chất phác thật thà của lão nông 71 tuổi làm du khách phương xa yêu mến'}
{'claim': 'Vân Nam sẽ tiếp tục phối hợp với các địa phương Việt Nam để giảm thiếu hợp tác ', 'evidence': 'bí thư vương ninh khẳng định vân nam sẽ tiếp tục phối hợp với các địa

In [ ]:


#Load data
df1 = pd.read_json('/content/drive/MyDrive/Dataset/output72.json')
df2 = pd.read_json('/content/drive/MyDrive/Dataset/output.json')
verdicts1 = []
verdicts2 = []
ids = []
for key, value in df1.items():
    verdict = value['evidence']
    verdicts1.append(verdict)
    ids.append(key)

for key, value in df2.items():
    verdict = value['evidence']
    verdicts2.append(verdict)

difVerdictID = []
for verdict1, verdict2, id in zip(verdicts1, verdicts2, ids):
  if(verdict1 != verdict2):
      difVerdictID.append(id)

In [ ]:
print(difVerdictID)

[39773, 44055, 40219, 40165, 39597, 45869, 37178, 40295, 38160, 39557, 40944, 40203, 39440, 38420, 37516, 40978, 40750, 37858, 39649, 39922, 44028, 38051, 41011, 44360, 40864, 39770, 40086, 40865, 39711, 40812, 39627, 40023, 39958, 44125, 40735, 38162, 39593, 40343, 39437, 40966, 37477, 40256, 37437, 39823, 39699, 40015, 39755, 40839, 40980, 40258, 37854, 39681, 40928, 39561, 40904, 39674, 40140, 39448, 40762, 40884, 40745, 40918, 45914, 40306, 40992, 37764, 40372, 39777, 39491, 39887, 40378, 39947, 44396, 37444, 37351, 45823, 37888, 45782, 39493, 40867, 39758, 39611, 37128, 40423, 40159, 40975, 39862, 40855, 40354, 40090, 39433, 40254, 40379, 40802, 39545, 40852, 40328, 39590, 41016, 40293, 39596, 39572, 39727, 39548, 40877, 39821, 44198, 45916, 40052, 40890, 40773, 39473, 40826, 40424, 37364, 39766, 39680, 40991, 40177, 39634, 39979, 44404, 44299, 40797, 40242, 39666, 40807, 40998, 44331, 40151, 39728, 40958, 44142, 40986, 43831, 37849, 40794, 39800, 44332, 45915]
